# Simplified Kyber - Beispiel mit Implementierung in Python

## 1.1 Was ist Kyber?
- Gewinner der NIST-Ausschreibung für PQC-Verfahren (Post-quantum cryptography)
    - Begründung für den Sieg: "comparatively small encryption keys \[...\] its speed of operation."
- Ist Teil der "Cryptographic Suite for Algebraic Lattices" (CRYSTALS), CRYSTALS-Dilithium (PQ-Signatur-Verfahren), ebenfalls Gewinner in der NIST-Ausschreibung
- Public Key Encryption System -> Key-Paare
- KEM -> Key encapsulation mechanism (Verschlüsselung von symmetrischen Schlüsseln)
    - Genaue Spezifikation heißt "IND-CCA2-secure key-encapsulation mechanism"
    - "Indistinguishability under adaptive chosen ciphertext attack"
- Sicherheit von Kyber basiert auf dem MLWE-Problem (Module learning with errors)
- Praktischer Einsatz möglich trotz längerer Schlüssel, immernoch weniger als andere PQC-Finalisten
    - Vgl. Classic McEliece -> Key size: > 1MB
- Die Operationen auf dem Polynomring (Multiplikation von großen Integern und Polynomen mit hohem Grad) lassen sich mit Hilfe von NTT (Number theoretic transforms) sehr stark optimieren 
    
### 1.2 Schlüssellängen
Version | Sicherheitslevel | Private Key | Public Key | Ciphertext 
:-: | :-: | :-: | :-: | :-: 
**Kyber512** | AES128 | 1632 | 800 | 768 
**Kyber768** | AES192 | 2400 | 1184 | 1088  
**Kyber1024** | AES256 |3168 | 1568 | 1568 
**RSA3072** | AES128 | 384 | 384 | 384 
**RSA15360** | AES256 | 1920 | 1920 | 1920 

*Größenangaben sind in Byte

## Mathematische Vorraussetzungen
- Alle Berechnungen finden in dem Polynomring $R = \mathbb{Z}[X]/(X^n + 1)$ und $R_q = \mathbb{Z_q}[X]/(X^n + 1)$, wobei $n = 256$ und $q = 7681$ ist.
- Jedes Polynom $p \in R_q$ kann wie folgt dargestellt werden:
$p = (\sum_{i=0}^{n-1} a_i * x^i) mod q$

## Module learning with errors
Einfach zu lösen ist das Problem:
Gegeben sei $A \in R_q^{kxk}$ mit zufällig gewählten Polynomen und $a_{i,j} \in R_q^{kx1}$ mit $i, j \in \{0, k-1\}$, dann lässt sich <br><br>
$\begin{pmatrix} a_{1,1} & \dots & a_{1,k} \\ \vdots & \ddots & \vdots \\ a_{k,1} & \dots & a_{k,k} \end{pmatrix}$ $\begin{pmatrix} s_1 \\ \vdots \\ s_k \end{pmatrix}$ $=$
$\begin{pmatrix} t_1 \\ \vdots \\ t_k \end{pmatrix}$ <br><br>
der Vektor $s$ mit dem gaußschen Eliminationsverfahren in $O(n^3)$ berechnen. Das Problem wird unverhältismäßig schwierig, wenn man noch einen Error-vektor $e$ mit "kleinen" Koeffizienten dazunimmt. Sei nun $e \in R_q^{kx1}$ mit zufällig gewählten "kleinen" Polynomen, dann nimmt man an, dass man $t$ mit <br><br>
$\begin{pmatrix} a_{1,1} & \dots & a_{1,k} \\ \vdots & \ddots & \vdots \\ a_{k,1} & \dots & a_{k,k} \end{pmatrix}$ $\begin{pmatrix} s_1 \\ \vdots \\ s_k \end{pmatrix}$ $+$
$\begin{pmatrix} e_1 \\ \vdots \\ e_k \end{pmatrix}$ $=$
$\begin{pmatrix} t_1 \\ \vdots \\ t_k \end{pmatrix}$ <br><br>
nicht mehr in polynomieller Laufzeit bestimmen kann. Über einen nicht trivialen Beweis lässt sich dieses Problem in das SVP (Shortest vector problem) überführen, von dem man weiß, dass dieses selbst für Quantencomputer NP-hard ist.

## Unterschiede zum regulärem Kyber:
- Kleinere Parameter für bessere Lesbarkeit
- Kompression des Chiffretexts - beinflusst nicht das zugrundeliegende Kryptosystem

# Key Generation
## 1. Modul $q$, mit $q$ ist eine Primzahl
> #### Warum ist $q = 3329$?
> - Initial wurde $q$ mit 7681 angesetzt um die Bedigung $q$ $mod$ $2n = 1$ zu erfüllen (Ermöglicht Optimierungen bei der Multiplikation)
> - In der 2. Runde wechselte man zu 3329, da man herausfand, dass auch $q$ $mod$ $n = 1$ reicht um gleiche, bzw. bessere Performance zu erreichen

Es wird $q = 17$ gesetzt und weil viel mit Polynomen gerechnet, Darstellung als Polynom: $f = x^4 + 1$. <br>
Für die Multiplikation von Zahlen wird $q$ und für die Multiplikation mit Polynomen wird $f$ verwendet.


In [1]:
from sympy import Poly
from sympy.abc import x, y
from sympy import GF
from sympy import QQ
import sympy as S 
import numpy as np
import math
import decimal

q = 17 # Modul
dom = GF(q, symmetric=False) # Galoisfeld als endlicher Zahlenkoerper
f = Poly(1 + x**4, domain=dom) # Binäre Representation von Q als Polynom
zero = Poly(0, x, domain=dom)
print('q: ', q)
print('f: ', f)
print(dom)

decimal.getcontext().rounding = decimal.ROUND_HALF_UP
def round(number):
    return int(decimal.Decimal(number).to_integral_value())

q:  17
f:  Poly(x**4 + 1, x, modulus=17)
GF(17)


## 2. Private key $s$
Sei $s = \begin{pmatrix} -x^3-x^2+x \\ -x^3-x \end{pmatrix}$

In [2]:
s = np.array([[Poly(-x**3 - x**2 + x)], [Poly(-x**3 - x)]])
print('s: ', s, '\ns-dimension: ', s.shape) # Spaltenvektor

s:  [[Poly(-x**3 - x**2 + x, x, domain='ZZ')]
 [Poly(-x**3 - x, x, domain='ZZ')]] 
s-dimension:  (2, 1)


## 3. Public key $(A, t)$
Sei $A = \begin{pmatrix} 6x^3+16x^2+16x+11 & 9x^3+4x^2+6x+3 \\ 5x^3+3x^2+10x+1 & 6x^3+x^2+9x+15 \end{pmatrix}$ <br><br>
$e = \begin{pmatrix} x^2 \\ x^2 - x\end{pmatrix}$ <br><br>
$t = As + e = \begin{pmatrix} 16x^3+15x^2+7 \\ 10x^3+12x^2+11x+6 \end{pmatrix}$

In [3]:
A = np.array([[Poly(6*x**3 + 16*x**2 + 16*x + 11, domain=dom), Poly(9*x**3 + 4*x**2 + 6*x + 3, domain=dom)],
                [Poly(5*x**3 + 3*x**2 + 10*x + 1, domain=dom), Poly(6*x**3 + 1*x**2 + 9*x + 15, domain=dom)]])
print('A: ', A, '\nA-dimension: ', A.shape)

e = np.array([[Poly(x**2)], [Poly(x**2 - x)]])
print('e: ', e, '\ne-dimension: ', e.shape)

A:  [[Poly(6*x**3 + 16*x**2 + 16*x + 11, x, modulus=17)
  Poly(9*x**3 + 4*x**2 + 6*x + 3, x, modulus=17)]
 [Poly(5*x**3 + 3*x**2 + 10*x + 1, x, modulus=17)
  Poly(6*x**3 + x**2 + 9*x + 15, x, modulus=17)]] 
A-dimension:  (2, 2)
e:  [[Poly(x**2, x, domain='ZZ')]
 [Poly(x**2 - x, x, domain='ZZ')]] 
e-dimension:  (2, 1)


In [4]:
def poly_mul(x, y): 
    rows = x.shape[0]
    cols = y.shape[1]
    y_rows = y.shape[0]
    result = np.full((rows, cols), zero) # Array mit 0 initialisieren
    for i in range(rows):
        for j in range(cols):
            for k in range(y_rows):
                result[i][j] = result[i][j].add((x[i][k].mul(y[k][j]))) # Skalarprodukt
            result[i][j] = result[i][j].rem(f) # Modulo Polynom F
            result[i][j] = Poly(result[i][j], domain=dom) # Modulo Primzahl Q
    return result

def poly_add(x, y):
    rows = x.shape[0]
    cols = y.shape[1]
    result = np.empty((rows, cols), Poly)
    for i in range(rows):
        for j in range(cols):
            result[i][j] = Poly(x[i][j].add(y[i][j]), domain=dom) # Modulo Primzahl Q
    return result

def poly_sub(x, y):
    rows = x.shape[0]
    cols = y.shape[1]
    result = np.empty((rows, cols), Poly)
    for i in range(rows):
        for j in range(cols):
            result[i][j] = Poly(x[i][j].sub(y[i][j]), domain=dom) # Modulo Primzahl Q
    return result

t = poly_mul(A, s)
t = poly_add(t, e)

pk = (A,t)
sk = s

print('A: ', A, '\nA-dimension: ', A.shape)
print('t: ', t, '\nt-dimension: ', t.shape)
print('s: ', s, '\ns-dimension: ', s.shape)


A:  [[Poly(6*x**3 + 16*x**2 + 16*x + 11, x, modulus=17)
  Poly(9*x**3 + 4*x**2 + 6*x + 3, x, modulus=17)]
 [Poly(5*x**3 + 3*x**2 + 10*x + 1, x, modulus=17)
  Poly(6*x**3 + x**2 + 9*x + 15, x, modulus=17)]] 
A-dimension:  (2, 2)
t:  [[Poly(16*x**3 + 15*x**2 + 7, x, modulus=17)]
 [Poly(10*x**3 + 12*x**2 + 11*x + 6, x, modulus=17)]] 
t-dimension:  (2, 1)
s:  [[Poly(-x**3 - x**2 + x, x, domain='ZZ')]
 [Poly(-x**3 - x, x, domain='ZZ')]] 
s-dimension:  (2, 1)


# Encryption
Wie üblich in Kryptosystemen, wird eine Nachricht $m$ mit einem Public Key verschlüsselt. Zusätzlich zu dem PK-Tupel $(A, t)$ werden auch noch ein Error- und Zufalls-Polynom-Vektoren $e_1$ und $r$ benötigt, welche für jede Verschlüsselung zufällig neu generiert werden. Außerdem braucht man noch ein Error-Polynom $e_2$. 

$r = \begin{pmatrix} -x^3 + x^2 \\ x^3 + x^2 - 1 \end{pmatrix}$ <br>
$e_1 = \begin{pmatrix} x^2 + x \\ x^2 \end{pmatrix}$ <br>
$e_2 = -x^3 -x^2$ <br>

In [5]:
r = np.array([[Poly(-x**3 + x**2)], [Poly(x**3 + x**2 - 1)]])
e_one = np.array([[Poly(x**2 + x)], [Poly(x**2)]])
e_two = Poly(-x**3 - x**2)
print('r: ', r, '\nr-dimension: ', r.shape)
print('e1: ', e_one, '\ne1-dimension: ', e_one.shape)
print('e2: ', e_two)

r:  [[Poly(-x**3 + x**2, x, domain='ZZ')]
 [Poly(x**3 + x**2 - 1, x, domain='ZZ')]] 
r-dimension:  (2, 1)
e1:  [[Poly(x**2 + x, x, domain='ZZ')]
 [Poly(x**2, x, domain='ZZ')]] 
e1-dimension:  (2, 1)
e2:  Poly(-x**3 - x**2, x, domain='ZZ')


Um eine Nachricht $m$ zu verschlüsseln bringt man diese erst in Binärdarstellung und verwendet die Bits als Koeffizienten. <br>
$m = 11$, $(11)_{10} = (1011)_{2}$ <br>
$m_b = 1x^3 + 0x^2 + 1x^2 + 1x^0 = x^3 + x + 1$ <br> <br>

In [6]:
m = 11
mb = np.array([int(x) for x in np.binary_repr(m)])
print(mb)

[1 0 1 1]


Die Nachricht als Binärpolynom muss nun noch um den Faktor $\lfloor \frac{q}{2} \rceil$ hochskaliert werden.  <br>
$m_{bs} = \lfloor \frac{q}{2} \rceil * m_b = 9 * m_b = 9x^3 + 9x + 9$ <br>
Warum man die Nachricht skaliert, wird bei der Entschlüsselung ersichtlich.

In [7]:
mbs = round(q/2) * mb
mbs_poly = Poly(mbs, x)
print(mbs_poly)

Poly(9*x**3 + 9*x + 9, x, domain='ZZ')


Das Ergebnis der eigentlichen Verschlüsselung, also der Chiffretext $c$, besteht aus dem Tupel $c = (u, v)$.<br>
$u^T = r^T A + e_1^T$<br>
$v = r^T t + e_2 + m_{bs}$<br>

$u = \begin{pmatrix} 11x^3 + 11x^2 + 10x + 3 \\ 4x^3 + 4x^2 + 13x + 11 \end{pmatrix}$ <br>
$v = 8x^3 + 6x^2 + 9x + 16$


In [8]:
u = poly_add(poly_mul(r.transpose(), A), e_one.transpose()).transpose()
print('u: ', u, '\nu-dimension: ', u.shape)
# [0][0], da das Polynom in einem 1x1 Vektor steht und man das tatsächlich Polynom zur weiteren Berechnung benötigt
v = np.array([[Poly(poly_mul(r.transpose(), t)[0][0] + e_two + mbs_poly, domain=dom)]])
print('v: ', v)

u:  [[Poly(11*x**3 + 11*x**2 + 10*x + 3, x, modulus=17)]
 [Poly(4*x**3 + 4*x**2 + 13*x + 11, x, modulus=17)]] 
u-dimension:  (2, 1)
v:  [[Poly(8*x**3 + 6*x**2 + 9*x + 16, x, modulus=17)]]


# Decryption
Die Entschlüsselung kann nun nur die Persion durchführen, welche den SK $s$ kennt. Um die verschlüsselte Nachricht $c = (u,v)$ nun zu entschlüsseln muss man folgendes berechnen\: <br>
$m_n = v - u * s$ <br>
$\Leftrightarrow m_n = r^T * t + e_2 + m_{bs} - (r^T * A + e_1^T)*s$ <br>
$\Leftrightarrow m_n = r^T * (A * s + e) + e_2 + m_{bs} - (r^T * A + e_1^T)*s$ <br>
$\Leftrightarrow m_n = r^T * A * s + r^T * e + e_2 + m_{bs} - r^T * A * s - e_1^T *s$ <br>
$\Leftrightarrow m_n = r^T * e + e_2 + m_{bs} - e_1^T *s$ <br>

Jetzt ist auch vielleicht ersichtlich warum man die Nachricht hochskaliert hat. In $m_n$ sind die Koeffizienten aller Terme ,außer $m_{bs}$, klein. Also kann man die Nachricht selbst mit dem Störsignal $r^T * e + e_2 - e_1^T *s$ wiederherstellen, indem man für jeden Koeffizienten schaut ob dieser näher an $\lfloor \frac{q}{2} \rceil$ oder an 0, bzw. $q$ ist.

In [9]:
mn = poly_sub(v, poly_mul(s.transpose(), u))[0][0]
print('mn: ', mn)

mn:  Poly(8*x**3 + 14*x**2 + 8*x + 6, x, modulus=17)


## Wiederherstellen der Nachricht
$\lfloor \frac{q}{2} \rceil = 9$ <br>
$q = 17$ <br>
$m_n = 8x^3 + 14x^2 + 8x + 6$

- 8, näher an 9 als an 0/q, nach 9 runden
- 14, näher an q als an 9, nach 0 runden
- 8, näher an 9 als an 0/q, nach 9 runden
- 6, näher an 9 als an 0/q, nach 9 runden

Man erhält also: <br>
$m_{bs} = 9x^3 + 9x + 9$ <br>
$m_{bs} = \lfloor \frac{q}{2} \rceil * m_b$ <br>
$m_b = 1x^3 + 0x^2 + 1x + 1$
Aus $m_b$ kann man nun die Bits der originalen Nachricht wieder ablesen und man erhält m = $(1011)_{10}$ = $(11)_2$.

# Zusammenfassung
- Vorteil von Gitter-basierten Encryption schemes ist, dass diese sehr schnell sind, besonders im Vgl. zu anderen PQC-Verfahren

# Quellen
- https://cryptopedia.dev/posts/kyber/
- https://pq-crystals.org/kyber/
- https://pq-crystals.org/kyber/data/kyber-specification-round2.pdf
- https://github.com/VadimLyubash/non-app-KyberSaber/blob/main/non-app.pdf

https://www.nist.gov/news-events/news/2022/07/nist-announces-first-four-quantum-resistant-cryptographic-algorithms
https://github.com/VadimLyubash/non-app-KyberSaber/blob/main/non-app.pdf
https://pq-crystals.org/kyber/data/slides-nistpqc19-schwabe.pdf
https://pq-crystals.org/kyber/data/slides-nistpqc18-schwabe.pdf
https://lukas-prokop.at/articles/2020-07-10-pqcrypto-performance
https://eprint.iacr.org/2016/504.pdf
https://pq-crystals.org/kyber/data/kyber-specification-round3-20210131.pdf